In [397]:
library(TTR)
library(quantmod)
library(dplyr)
library(FKF)
library(NMOF)
library(data.table)
library(plotly)
library(stringr)
library(tidyverse)
library(doParallel)

In [398]:
COURTAGE=0.00089

In [399]:
df <- read.csv('http://r-forge.r-project.org/scm/viewvc.php/*checkout*/pkg/fBasics/data/DowJones30.csv?revision=1&root=rmetrics&pathrev=1',sep=';');
head(df)

,X.Y..m..d,AA,AXP,T,BA,CAT,C,KO,DD,EK,⋯,MCD,MRK,MSFT,MMM,MO,PG,SBC,UTX,WMT,DIS
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1990-12-31,5.92,4.70,14.67,21.77,9.30,1.87,9.88,13.15,23.02,⋯,7.08,11.83,2.08,31.41,10.55,20.91,9.75,9.31,7.05,7.86
2,1991-01-02,5.92,4.70,14.67,21.77,9.30,1.87,9.88,13.15,23.02,⋯,7.08,11.83,2.08,31.41,10.55,20.91,9.75,9.31,7.05,7.86
3,1991-01-03,5.88,4.73,14.73,21.71,9.15,1.87,9.60,12.83,22.95,⋯,6.99,11.45,2.09,30.90,10.27,20.70,9.64,9.19,7.05,7.82
4,1991-01-04,5.76,4.73,14.79,22.50,9.00,1.92,9.85,13.06,22.74,⋯,7.02,11.27,2.11,30.80,10.24,20.48,9.56,9.24,6.97,7.79
5,1991-01-07,5.72,4.58,14.67,21.65,8.87,1.89,9.69,12.83,22.11,⋯,6.80,10.97,2.08,30.43,10.08,20.05,9.40,8.95,6.82,7.58
6,1991-01-08,5.63,4.53,14.73,21.34,9.05,1.85,9.60,12.60,22.18,⋯,6.83,11.01,2.04,29.65,10.11,20.02,9.32,8.78,6.85,7.41


In [400]:
stock_names <- c(tail(colnames(df),-1),'AAPL','LMT', 'ALV')
stock_names <- stock_names[!stock_names %in% c('DD', 'EK', 'HWP', 'SBC', 'UTX')]

stock_names <- c(stock_names, c('AMZN', 'TSLA', 'GOOGL', 'GOOG', 'UNH', 'JNJ', 'XOM'))
stock_names <- c(stock_names, c('META', 'NVDA', 'V', 'HD', 'CVX', 'ABBV', 'MA', 'PFE'))
stock_names <- c(stock_names, c('LLY', 'PEP', 'KO', 'MRK', 'BAC', 'COST', 'TMO', 'WMT', 'AVGO', 'MCD'))
stock_names <- c(stock_names, c('ABT', 'DHR', 'CSCO', 'VZ', 'ACN', 'NEE', 'WFC', 'BMY', 'TXN'))
stock_names

[1] "AA"    "AXP"   "T"     "BA"    "CAT"   "C"     "KO"    "XOM"   "GE"   
[10] "GM"    "HD"    "HON"   "INTC"  "IBM"   "IP"    "JPM"   "JNJ"   "MCD"  
[19] "MRK"   "MSFT"  "MMM"   "MO"    "PG"    "WMT"   "DIS"   "AAPL"  "LMT"  
[28] "ALV"   "AMZN"  "TSLA"  "GOOGL" "GOOG"  "UNH"   "JNJ"   "XOM"   "META" 
[37] "NVDA"  "V"     "HD"    "CVX"   "ABBV"  "MA"    "PFE"   "LLY"   "PEP"  
[46] "KO"    "MRK"   "BAC"   "COST"  "TMO"   "WMT"   "AVGO"  "MCD"   "ABT"  
[55] "DHR"   "CSCO"  "VZ"    "ACN"   "NEE"   "WFC"   "BMY"   "TXN"

In [469]:
library(rvest)
url <- "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
whole_stock_names <- url %>%
  read_html %>%
  html_nodes(css='#constituents') %>%
  html_table() %>% data.frame %>% pull(Symbol)

In [507]:
length(whole_stock_names)

[1] 503

In [485]:
# test if we can get data from yahoo
data_ok <- function(name) {
  res <- TRUE
  data <- tryCatch(getSymbols(name,src='yahoo',auto.assign=FALSE), error=function(e) {res <<- FALSE})
  if (!res) {
    return(res)
  } else {
    return(length(data)>3000)
  }
}

fetch_ok <- lapply(whole_stock_names, data_ok)



Warning message:
“BRK.B download failed; trying again.”
Warning message:
“BF.B download failed; trying again.”
Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
“incomplete final line found by readTableHeader on 'https://query1.finance.yahoo.com/v7/finance/download/DD?period1=1167609600&period2=1664496000&interval=1d&events=history'”


In [488]:
stock_names <- whole_stock_names[fetch_ok==TRUE]

In [510]:
stock_name_list <- split(stock_names, ceiling(seq_along(stock_names)/8))

In [514]:
length(stock_name_list)

[1] 63

[1] 1
[1] "MMM"  "AOS"  "ABT"  "ABBV" "ABMD" "ACN"  "ATVI" "ADM" 
[1] 2
[1] "ADBE" "ADP"  "AAP"  "AES"  "AFL"  "A"    "APD"  "AKAM"
[1] 3
[1] "ALK"   "ALB"   "ARE"   "ALGN"  "ALLE"  "LNT"   "ALL"   "GOOGL"
[1] 4
[1] "GOOG" "MO"   "AMZN" "AMCR" "AMD"  "AEE"  "AAL"  "AEP" 
[1] 5
[1] "AXP"  "AIG"  "AMT"  "AWK"  "AMP"  "ABC"  "AME"  "AMGN"
[1] 6
[1] "APH"  "ADI"  "ANSS" "AON"  "APA"  "AAPL" "AMAT" "APTV"
[1] 7
[1] "ANET" "AJG"  "AIZ"  "T"    "ATO"  "ADSK" "AZO"  "AVB" 
[1] 8
[1] "AVY"  "BKR"  "BALL" "BAC"  "BBWI" "BAX"  "BDX"  "WRB" 
[1] 9
[1] "BBY"  "BIO"  "TECH" "BIIB" "BLK"  "BK"   "BA"   "BKNG"
[1] 10
[1] "BWA"  "BXP"  "BSX"  "BMY"  "AVGO" "BR"   "BRO"  "CHRW"
[1] 11
[1] "CDNS" "CZR"  "CPT"  "CPB"  "COF"  "CAH"  "KMX"  "CCL" 
[1] 12
[1] "CARR" "CTLT" "CAT"  "CBOE" "CBRE" "CDW"  "CE"   "CNC" 
[1] 13
[1] "CNP"  "CDAY" "CF"   "CRL"  "SCHW" "CHTR" "CVX"  "CMG" 
[1] 14
[1] "CB"   "CHD"  "CI"   "CINF" "CTAS" "CSCO" "C"    "CFG" 
[1] 15
[1] "CTXS"  "CLX"   "CME"   "CMS"   "KO"    "CTSH"  "CL"

In [490]:
kalman <- function(df, HHt_val, GGt_val) {
    df$mean <- (df$high + df$low)/2
    y <- as.numeric(df$mean)
    dt <- ct <- matrix(0)
    Tt <- matrix(1)
    a0 <- y[1] # Estimation of the first year flow
    P0 <- matrix(0.0003) # Variance of 'a0'
    Zt <- matrix(1)
    HHt <- matrix(0)
    res <- fkf(HHt = matrix(HHt_val), GGt = matrix(HHt_val), yt = rbind(y), Zt=Zt,a0 = a0, P0 = P0, dt = dt, ct = ct,
        Tt = Tt)

    len <- length(y)

    resid <- y - res$at[1,1:len]
    #df <- data.frame(y, res$at[1,1:len], resid[1:len], df$high, df$low, df$close, df$open, shift(df$close))
    df <- df %>% mutate(mean=y, predict=res$at[1,1:len], resid=resid[1:len], last_close=shift(df$close))
    df$high_resid <- df$high - df$predict
    df$low_resid <- df$low - df$predict
    cal_resid_max <- function(x) {
        high_resid <- x['high_resid']
        low_resid <- x['low_resid']
        if (abs(high_resid) >  abs(low_resid)) {
            return(high_resid)
        } else {
            return(low_resid)
        }
    }

    df$max_resid <- apply(df, 1, cal_resid_max)
    return(df)
}


if (hold) {
    if (up) {
        if (stop_loss) sell # no trade today
        else if(sell_point) sell
    } else {
        if (sell_point) sell {
            if (buy_point) buy if (stop_loss) sell
        }
    }
} else {
    if (up) {
        if (buy_point) {
            buy 
            if (stop_loss) sell
            elseif(sell_point) sell
        }
    } else {
        if (buy_point) {
            buy 
            if (stop_loss) sell
        }
        
    }
}





In [493]:
get_daily_profit <- function(par, df, verbose=TRUE, plot=FALSE, same_day_trade_allowed=TRUE, courtage=COURTAGE) {
    BUY_SD_Z_SCORE <- par[1]
    SELL_SD_Z_SCORE <- par[2]
    HHt_val <- par[3]
    GGt_val <- par[4]
    if (par[5]>1 || par[5] < 0) {
      if (verbose) print(str_glue('warning stop:loss {par[5]} is invalid'))
      return(NULL)
    }
    log_stop_loss <- log(par[5])  # always negative
    hold <- 0
    starts <- c()
    ends <- c()
    courtage_log <- log(1-courtage)  #default -0.0015011
    buy_point <- 0
    
    daily_trade <- function(x, max_resid_mean, max_resid_std) {
      INVALID <-  (c(0, -999999, 0))
      low_resid <- x['low_resid']
      high_resid <- x['high_resid']
      predict <- x['predict']
      close <- x['close']
      open <- x['open']
      low <- x['low']
      high <- x['high']
      last_close <- x['last_close']
      if(is.na(last_close)) {
        last_close = open
      }
      row_id <- x['row_id']
      
      profit <- 0
      trade_times <- 0
      #low_z <- (low_resid - max_resid_mean) / max_resid_std
      #high_z <- (high_resid - max_resid_mean) / max_resid_std
      low_buy_threshold <- (predict + max_resid_mean) + BUY_SD_Z_SCORE * max_resid_std
      high_sell_threshold <- (predict + max_resid_mean) + SELL_SD_Z_SCORE * max_resid_std
      low_stop_loss_threshold <- buy_point + log_stop_loss
      
      # check to prevent extreme value
      if (hold && low_stop_loss_threshold > high_sell_threshold) {
        if (verbose) print(str_glue('warning stop_loss {low_stop_loss_threshold} higher than high_sell {high_sell_threshold}'))
        return(INVALID)
      }
      up <- close > open
      
      buy_at <- function(price) {
        hold <<- 1
        buy_point <<- price
        trade_times <<- trade_times+1
        low_stop_loss_threshold <<- buy_point + log_stop_loss
        if (verbose) {
          print(str_glue('BUY: row_id {row_id} up:{up} low:{low} buy:{buy_point} trade_times:{trade_times}'))
        }
          
      }

      sell_at <- function(price) {
        hold <<- 0
        trade_times <<- trade_times+1
        if (verbose) {
          if (price == low_stop_loss_threshold) {
            print(str_glue('SELL(STOP): row_id {row_id} up:{up} low:{low} stop_loss:{low_stop_loss_threshold} trade_times:{trade_times}'))    
          } else {
            print(str_glue('SELL: row_id {row_id} up:{up} high:{high} price:{price} sell:{high_sell_threshold} trade_times:{trade_times}'))    
          }
        }
      }      

      if (hold) {
        # the max (low_buy_threshold, low_stop_loss_threshold) come first
        if (up) {
          if (low < low_stop_loss_threshold) {
            sell_at(low_stop_loss_threshold)
            profit <- low_stop_loss_threshold - last_close + courtage_log
          } else if(high > high_sell_threshold) {
            sell_at(high_sell_threshold)
            profit <- high_sell_threshold - last_close + courtage_log
          } 
        } else {
          if(high > high_sell_threshold) {
            sell_at(high_sell_threshold)
            profit <- high_sell_threshold - last_close + courtage_log
            if (low < low_buy_threshold) {
              buy_at(low_buy_threshold)
              profit <- profit + courtage_log
              low_stop_loss_threshold <- buy_point + log_stop_loss
              if (low < low_stop_loss_threshold) {
                sell_at(low_stop_loss_threshold)
                profit <- profit + low_stop_loss_threshold - buy_point + courtage_log
              } else {
                profit <- profit + close - buy_point
              }
            }
          } else if (low < low_stop_loss_threshold) {
            sell_at(low_stop_loss_threshold)
            profit <- low_stop_loss_threshold - last_close + courtage_log
          } 
        }    
      } else {
        if(up) {
          if (low < low_buy_threshold) {
            buy_at(low_buy_threshold)
            profit <- courtage_log
            low_stop_loss_threshold <- buy_point + log_stop_loss
            if (low < low_stop_loss_threshold) {
              sell_at(low_stop_loss_threshold)
              profit <- profit + low_stop_loss_threshold - buy_point + courtage_log
            } else if(high > high_sell_threshold) {
              profit <- profit + high_sell_threshold - buy_point + courtage_log
            } else {
              profit <- profit + close - buy_point
            }
          }
        } else {
          if (low < low_buy_threshold) {
            buy_at(low_buy_threshold)
            profit <- courtage_log
            low_stop_loss_threshold <- buy_point + log_stop_loss
            if (low < low_stop_loss_threshold) {
              sell_at(low_stop_loss_threshold)
              profit <- profit + low_stop_loss_threshold - buy_point + courtage_log
            } else {
              profit <- profit + close - buy_point
            }
          }
        }
      }
      if (trade_times > 0 && verbose) {
        print(str_glue('DAY: row_id: {row_id} open:{open} close:{close} last_close:{last_close} profit: {profit} trade_times: {trade_times} hold: {hold}'))
      }
      return(c(hold, profit, trade_times))
    }
    

    df <- kalman(df, HHt_val, GGt_val)
    
    max_resid_std <- sd(df$max_resid)
    max_resid_mean <- mean(df$max_resid)
    result <- apply(df %>% mutate(row_id=row_number()) , 1, function(x) daily_trade(x, max_resid_mean, max_resid_std))
    df$hold=t(result)[,1]
    df$trade_profit=t(result)[,2]
    df$trade_times=t(result)[,3]
    return(df)
    #return(cbind(df, df_result))
}

test <- function() {
  df <- data.frame(getSymbols('TSLA',src='yahoo',auto.assign=FALSE))
  df <- left_merge_with_rowname(df, sek_df)
  df$SEK.X.Mean <- na.approx(df$SEK.X.Mean)
  df[,1:4] <- df[,1:4] * df$SEK.X.Mean
  df <- data.frame(log(df)) %>% select(-last_col())
  #colnames(df) <- c('open', 'high', 'low', 'close', 'volume', 'adjusted')
  df <- df %>% set_colnames(c('open', 'high', 'low', 'close', 'volume', 'adjusted'))
  df <- get_daily_profit(c(-2.56992,0.53008,0.01508,0.14708,0.75), df=df, verbose=TRUE, same_day_trade_allowed=TRUE)  
}


In [494]:
result_summary <- function(df, courtage=COURTAGE, verbose=FALSE) {
  if (is.null(df)) {
    # invalid
    return(-999999)
  }
  
  if(verbose) {
    summary <- df %>% filter(trade_profit != 0) %>% summarise(sum=sum(trade_profit), times=sum(trade_times), n=n())
    avg_profit <- summary$sum / summary$n
    len <- dim(df)[1]
    avg_profit_no_model <- (df$mean[len] - df$mean[1])/len    
  }


  
  summary2 <- df %>% mutate(id=row_number()) %>% mutate(range=cut(id, seq(0, max(id)+252, 252))) %>% group_by(range) %>% summarise(sum=sum(trade_profit)) 
  if (verbose) {
    
  }
  annual_mean <- mean(summary2$sum)
  annual_sd <- sd(summary2$sum)
  if (verbose) {
    print(str_glue('avg_profit:{avg_profit}, nomodel_profit:{avg_profit_no_model}, sum:{summary$sum}, trade_times:{summary$times} courtage: {courtage}'))
    print(str_glue('annaul_mean:{annual_mean} annual_sd:{annual_sd}'))
  }
  return(annual_mean - 0.5*annual_sd)
  
}

test <- function() {
  
  result_summary(df, verbose=TRUE)
}

#test()


In [495]:
set_colnames <- function(df, names) {
    colnames(df) <- names
    return(df)
}

In [496]:
merge_with_rowname <- function(x,y)
  transform(merge(x, y,all=TRUE,by=0), row.names=Row.names, Row.names=NULL)


left_merge_with_rowname <- function(x,y)
  transform(merge(x, y,all.x=TRUE,by=0), row.names=Row.names, Row.names=NULL)

In [497]:
data_prep <- function(stock_name, sek_df) {
  df <- data.frame(getSymbols(stock_name,src='yahoo',auto.assign=FALSE))
  df <- left_merge_with_rowname(df, sek_df)
  df$SEK.X.Mean <- na.approx(df$SEK.X.Mean)
  df[,1:4] <- df[,1:4] * df$SEK.X.Mean
  df <- data.frame(log(df)) %>% select(-last_col())
#colnames(df) <- c('open', 'high', 'low', 'close', 'volume', 'adjusted')
  df <- df %>% set_colnames(c('open', 'high', 'low', 'close', 'volume', 'adjusted'))
  return(df)
}


optimize <- function(stock_name, sek_df) {
  df <- data_prep(stock_name, sek_df)
  my_func <- function(par) {
      df <- get_daily_profit(par, df=df, verbose=FALSE, same_day_trade_allowed=TRUE)
      return(-result_summary(df=df, verbose=FALSE))
  }
  fit <- optim(c(-3,0.1,0.005,0.005,0.90), fn=my_func, control=list(maxit=800))
  fit$name <- stock_name
  return(fit)
}

print_vector <- function(v) {
  print(paste(shQuote(v), collapse=", "))
}

test <- function() {
  name <- 'V'
  fit <- optimize(name, sek_df)
  print(fit)
  df <- data_prep(name, sek_df)
  df <- get_daily_profit(fit$par, df=df, verbose=FALSE, same_day_trade_allowed=TRUE)
  result_summary(df=df, verbose=TRUE)
  print_vector(fit$par)
}

#test()


In [502]:
cut(length(stock_names),4)

[1] (497.8,498]
Levels: (497.5,497.8] (497.8,498] (498,498.2] (498.2,498.5]

pars <- list()
for (name in stock_names) {
    print(name)
    fit <- optimize(name, sek_df)
    pars[[name]] <- fit
}

In [ ]:
# parrellel test

my.cluster <- parallel::makeCluster(
  4, 
  type = "FORK"
)

doParallel::registerDoParallel(my.cluster)

for (i in seq_along(stock_name_list)) {
  stock_names <- stock_name_list[[i]]
  print(stock_names)
  pars <- foreach(name = stock_names) %dopar% {
    optimize(name, sek_df)
  }
  
  saveRDS(pars, file=str_glue("parameters_par_{i}.rds"))
  print("Finished")
  print(pars)
}

In [412]:
pars <- foreach(name = stock_names) %dopar% {
  optimize(name, sek_df)
}

stopCluster(my.cluster)
saveRDS(pars, file="parameters_par.rds")

In [516]:
pars <- readRDS(file="parameters_par.rds")

In [415]:
df_pars <- data.frame(do.call(rbind, pars))
dim(df_pars)

df_pars$new_value <- as.numeric(df_pars$value)
order(df_pars$new_value)
df_pars[order(df_pars$new_value),]


print_vector(df_pars[order(df_pars$new_value),]$name)

[1] 62  6

[1] 40 43 26 38 32 30 41  1 52 42  4 28  9  5 15 11 39 33 25  8 35 62 12 29 31
[26] 59  2 50 13  3 55 57 21  6 37 19 47 60 45 58 18 53 56 44 17 34 16 49 14 10
[51]  7 46 61 22 23 27 20 48 54 24 51 36

,par,value,counts,convergence,message,name,new_value
,<list>,<list>,<list>,<list>,<list>,<list>,<dbl>
40,"-0.06109344, 0.52237406, -2.33038806, 1.96881155, 0.12339898",-0.3836113,"423, NA",0,NULL,CVX,-0.383611304
43,"-0.1173417, 0.5551616, 1.4784009, -2.1521286, 0.5390080",-0.3833213,"428, NA",0,NULL,PFE,-0.383321322
26,"-0.01854904, 0.84614847, 0.84045590, -2.25730032, 0.22373561",-0.3225493,"440, NA",0,NULL,AAPL,-0.322549285
38,"-1.5925644, 1.1966535, -1.8700545, 1.0278844, 0.2421636",-0.08753738,"372, NA",0,NULL,V,-0.087537379
32,"-1.3770202, 1.6148199, -0.7944707, -0.1027833, 0.8774182",-0.08500935,"801, NA",1,NULL,GOOG,-0.085009355
30,"-2.4603697, 0.5856582, -0.1815120, 0.1898431, 0.1751482",-0.07893841,"447, NA",0,NULL,TSLA,-0.078938411
41,"-2.5982014, 0.4286843, 0.1150031, 0.1704271, 0.3900175",-0.03065321,"369, NA",0,NULL,ABBV,-0.030653210
1,"-2.4312759, 0.6389227, -0.1481891, 0.1262528, 0.2302657",-0.02792838,"366, NA",0,NULL,AA,-0.027928376
52,"-2.6547614, 1.1982277, 0.6257247, -0.6810316, 0.2283132",-0.02645233,"362, NA",0,NULL,AVGO,-0.026452331


[1] "'CVX', 'PFE', 'AAPL', 'V', 'GOOG', 'TSLA', 'ABBV', 'AA', 'AVGO', 'MA', 'BA', 'ALV', 'GE', 'CAT', 'IP', 'HD', 'HD', 'UNH', 'DIS', 'XOM', 'XOM', 'TXN', 'HON', 'AMZN', 'GOOGL', 'NEE', 'AXP', 'TMO', 'INTC', 'T', 'DHR', 'VZ', 'MMM', 'C', 'NVDA', 'MRK', 'MRK', 'WFC', 'PEP', 'ACN', 'MCD', 'MCD', 'CSCO', 'LLY', 'JNJ', 'JNJ', 'JPM', 'COST', 'IBM', 'GM', 'KO', 'KO', 'BMY', 'MO', 'PG', 'LMT', 'MSFT', 'BAC', 'ABT', 'WMT', 'WMT', 'META'"


In [162]:

apply_model <- function(name, par, verbose=FALSE) {
    df1 <- getSymbols(name,src='yahoo',auto.assign=FALSE)
    df1 <- data.frame(log(df1))
    #colnames(df) <- c('open', 'high', 'low', 'close', 'volume', 'adjusted')

    df1 <- df1 %>% set_colnames(c('open', 'high', 'low', 'close', 'volume', 'adjusted'))

    df1 <- get_daily_profit(par=par, df=df1, verbose=verbose,same_day_trade_allowed=TRUE)
    result_summary(df=df1, verbose=verbose)
    df1 <- df1 %>% rename_all(function(x) paste0(name,'.', x))
    return(df1)
}


In [416]:
chosen_names <- c('CVX', 'PFE', 'AAPL', 'V', 'GOOG', 'TSLA', 'ABBV', 'AA', 'AVGO', 'MA')
profits_names <- paste0(chosen_names, '.trade_profit')
profits_names

[1] "CVX.trade_profit"  "PFE.trade_profit"  "AAPL.trade_profit"
 [4] "V.trade_profit"    "GOOG.trade_profit" "TSLA.trade_profit"
 [7] "ABBV.trade_profit" "AA.trade_profit"   "AVGO.trade_profit"
[10] "MA.trade_profit"

In [417]:

get_par_by_name <- function(pars, name) {
  for (i in seq(length(pars))) {
    if (pars[[i]]$name==name)
      return(pars[[i]]$par)
  }
  return(NA)
}

get_par_by_name(pars,'GE')

[1] -3.5398548  0.1377492  0.9843371  0.2643794  0.3839306

In [418]:


all_df <- lapply(chosen_names, function(x) apply_model(x, par=get_par_by_name(pars,x))) %>% reduce(merge_with_rowname)

In [419]:
head(all_df)

,CVX.open,CVX.high,CVX.low,CVX.close,CVX.volume,CVX.adjusted,CVX.mean,CVX.predict,CVX.resid,CVX.last_close,⋯,MA.mean,MA.predict,MA.resid,MA.last_close,MA.high_resid,MA.low_resid,MA.max_resid,MA.hold,MA.trade_profit,MA.trade_times
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2007-01-03,4.287304,4.293469,4.256322,4.262257,16.35865,3.659893,4.274895,4.274895,0.0000000000,NA,⋯,2.278641,2.278641,0.00000000,NA,0.02073924,-0.020739238,-0.02073924,0,0,0
2007-01-04,4.261270,4.261975,4.242765,4.252487,16.19740,3.650123,4.252370,4.274895,-0.0225252211,4.262257,⋯,2.287680,2.278641,0.00903966,2.266025,0.04227533,-0.024196006,0.04227533,0,0,0
2007-01-05,4.252487,4.264369,4.250208,4.256322,16.07915,3.653957,4.257288,4.263633,-0.0063449175,4.252487,⋯,2.308441,2.283162,0.02527835,2.313921,0.04118426,0.009372432,0.04118426,0,0,0
2007-01-08,4.263525,4.270536,4.248495,4.268998,16.05995,3.666634,4.259516,4.259826,-0.0003106314,4.256322,⋯,2.308685,2.298330,0.01035510,2.313129,0.02650558,-0.005795368,0.02650558,0,0,0
2007-01-09,4.262398,4.267457,4.246350,4.257455,16.16689,3.655091,4.256904,4.259635,-0.0027316198,4.268998,⋯,2.333530,2.304703,0.02882740,2.313327,0.05379018,0.003864619,0.05379018,0,0,0
2007-01-10,4.244917,4.252914,4.233091,4.240031,16.29513,3.637667,4.243003,4.257948,-0.0149452813,4.257455,⋯,2.348852,2.322508,0.02634417,2.358398,0.04305217,0.009636172,0.04305217,0,0,0


In [420]:
get_first_profit <- function(x) {
  profits <- as.numeric(x[profits_names])
  index <- which(profits!=0 & !is.na(profits))[1]
  stock_name <- chosen_names[index]
  first_non_zero <- profits[index]
  
  if (!is.na(index)) {
    return(c(profits[index], stock_name))
  }
  return(c(0, ''))
}

profit_and_name <- apply(all_df, 1, get_first_profit)
dim(t(profit_and_name))
all_df$trade_profit <- as.numeric(t(profit_and_name)[,1])
all_df$name <- t(profit_and_name)[,2]

[1] 3964    2

In [421]:
head(all_df,20)

,CVX.open,CVX.high,CVX.low,CVX.close,CVX.volume,CVX.adjusted,CVX.mean,CVX.predict,CVX.resid,CVX.last_close,⋯,MA.resid,MA.last_close,MA.high_resid,MA.low_resid,MA.max_resid,MA.hold,MA.trade_profit,MA.trade_times,trade_profit,name
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
2007-01-03,4.287304,4.293469,4.256322,4.262257,16.35865,3.659893,4.274895,4.274895,0.0000000000,NA,⋯,0.000000000,NA,0.020739238,-0.0207392383,-0.02073924,0,0,0,-0.0120683945,CVX
2007-01-04,4.261270,4.261975,4.242765,4.252487,16.19740,3.650123,4.252370,4.274895,-0.0225252211,4.262257,⋯,0.009039660,2.266025,0.042275326,-0.0241960060,0.04227533,0,0,0,0.0091229681,PFE
2007-01-05,4.252487,4.264369,4.250208,4.256322,16.07915,3.653957,4.257288,4.263633,-0.0063449175,4.252487,⋯,0.025278345,2.313921,0.041184259,0.0093724315,0.04118426,0,0,0,0.0010502496,PFE
2007-01-08,4.263525,4.270536,4.248495,4.268998,16.05995,3.666634,4.259516,4.259826,-0.0003106314,4.256322,⋯,0.010355104,2.313129,0.026505576,-0.0057953684,0.02650558,0,0,0,0.0000000000,
2007-01-09,4.262398,4.267457,4.246350,4.257455,16.16689,3.655091,4.256904,4.259635,-0.0027316198,4.268998,⋯,0.028827400,2.313327,0.053790180,0.0038646194,0.05379018,0,0,0,0.0287654493,AAPL
2007-01-10,4.244917,4.252914,4.233091,4.240031,16.29513,3.637667,4.243003,4.257948,-0.0149452813,4.257455,⋯,0.026344170,2.358398,0.043052169,0.0096361719,0.04305217,0,0,0,0.0000000000,
2007-01-11,4.238733,4.264650,4.226542,4.229604,16.33379,3.627240,4.245596,4.248712,-0.0031164218,4.240031,⋯,0.019810565,2.348896,0.041036042,-0.0014149127,0.04103604,0,0,0,0.0043856654,CVX
2007-01-12,4.235555,4.257313,4.235555,4.253483,16.05035,3.651118,4.246434,4.246786,-0.0003520895,4.229604,⋯,0.010230294,2.356126,0.021546194,-0.0010856063,0.02154619,0,0,0,0.0007573339,PFE
2007-01-16,4.249637,4.258446,4.237434,4.243770,15.92956,3.641405,4.247940,4.246569,0.0013713802,4.253483,⋯,0.003882007,2.353753,0.017272718,-0.0095087033,0.01727272,0,0,0,0.0245968203,AAPL


In [422]:
profits <- all_df$trade_profit
sum(profits[profits!=0])
length(profits[profits!=0])

[1] 13.49049

[1] 2234

In [438]:
add_range <- function(df){
  df %>% mutate(id=row_number()) %>% mutate(range=cut(id, seq(0, dim(df)[1]+252, 252)))
}

r <- all_df['trade_profit'] %>% add_range %>% filter(trade_profit != 0) %>% group_by(range) %>% summarise(sum=sum(trade_profit), n=n())
r
r %>% summarise(mean=mean(sum), std=sd(sum))

range,sum,n
<fct>,<dbl>,<int>
"(0,252]",0.9566235,159
"(252,504]",0.9099806,189
"(504,756]",0.8861314,162
"(756,1.01e+03]",0.7687449,145
"(1.01e+03,1.26e+03]",0.7328430,148
"(1.26e+03,1.51e+03]",0.8376528,125
"(1.51e+03,1.76e+03]",0.8208440,120
"(1.76e+03,2.02e+03]",0.8141209,109
"(2.02e+03,2.27e+03]",0.6847856,131


mean,std
<dbl>,<dbl>
0.8431557,0.2035191


In [423]:
all_df %>% select(trade_profit, name) %>% filter(trade_profit!=0) %>% group_by(name) %>% summarise(sum=sum(trade_profit), n=n())

name,sum,n
<chr>,<dbl>,<int>
AA,0.2973545,15
AAPL,2.5944090,273
ABBV,0.2435094,11
AVGO,0.1737956,8
CVX,5.8851236,1125
GOOG,0.1840782,46
MA,0.1404158,10
PFE,3.2172431,681
TSLA,0.3951285,19


In [ ]:
# TODO: another optimization, what is the good priorities of the portofolio? make a sharpe analysis.

In [204]:
profits

[1]  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
   [6]  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  1.243782e-03
  [11] -1.918628e-02  3.928092e-03 -1.836047e-02 -3.729607e-03  4.094139e-02
  [16] -2.224656e-02  1.575344e-02 -6.819717e-03  3.749512e-03  1.442055e-02
  [21] -4.017828e-02 -5.191043e-04 -3.023440e-02  9.204812e-03 -3.122647e-03
  [26]  2.167749e-03 -1.959496e-02 -7.824603e-03  1.765846e-03  1.476741e-02
  [31] -9.618394e-03  1.818796e-02  4.585856e-03  7.932867e-03 -2.109357e-05
  [36] -1.105164e-02 -1.216413e-02 -3.537632e-02  1.514060e-03 -2.718110e-03
  [41] -2.153633e-02  5.161356e-03  3.695465e-02 -4.183172e-03 -2.021169e-03
  [46] -3.878051e-03  3.944006e-03 -2.611034e-02  1.115574e-02 -4.048360e-03
  [51] -1.204015e-02  1.436829e-02 -4.369710e-03  2.499494e-02  1.195329e-02
  [56] -4.546620e-04  6.840543e-03 -2.972140e-03 -3.760085e-03 -2.080679e-03
  [61] -6.005995e-03  8.072121e-04  3.022369e-02 -3.348852e-03  1.039818e-03
  [66] -7.023404e-03 -3.658948e-03 -4.231871e-03  6.137885e-03 -2.356242e-03
  [71]  1.696902e-02 -3.104334e-03  6.766453e-03 -9.201680e-03  2.270220e-02
  [76] -7.071822e-03 -3.240588e-03  9.628219e-04  6.651592e-03 -4.519995e-03
  [81] -1.605685e-02 -5.061810e-03 -6.889315e-03  1.586807e-02 -4.468688e-03
  [86] -8.205626e-03 -9.849216e-04  5.213421e-03 -1.671866e-02  1.135531e-02
  [91] -1.068377e-02 -8.219393e-03  3.140133e-02 -3.497363e-03 -1.359829e-03
  [96]  5.951741e-04  1.111522e-02 -3.979702e-03  7.592203e-04  1.918939e-02
 [101]  7.397304e-03  2.331425e-02 -1.384867e-03  4.988477e-03  1.324126e-02
 [106]  2.294647e-02 -1.137805e-03 -6.174338e-03  8.345170e-04 -8.083207e-03
 [111] -1.293180e-02  9.306741e-04 -4.761540e-03  6.047205e-03  1.823593e-02
 [116] -1.728985e-03 -8.474309e-03  8.085389e-03  2.092290e-02  4.637012e-03
 [121]  5.370267e-03 -7.515032e-03 -2.435119e-03 -4.409598e-03  1.458604e-02
 [126]  7.438586e-03  1.355074e-02 -4.125640e-03  5.841268e-03  1.436583e-03
 [131]  2.077532e-03  1.578293e-03  1.244672e-02  1.502094e-03  3.628170e-03
 [136] -9.959301e-03 -1.657420e-03 -5.329178e-02 -1.473937e-02  2.903050e-03
 [141] -8.283225e-03 -3.458603e-03  7.628323e-03  8.210160e-03 -1.190923e-02
 [146]  5.748190e-03 -3.769734e-03 -1.579897e-02  1.382052e-02  1.173480e-02
 [151]  1.873736e-02 -2.124035e-02  1.979678e-03 -4.848749e-04 -1.347546e-02
 [156] -2.196577e-02 -1.219342e-02  1.718542e-02 -4.248667e-03  1.730208e-02
 [161]  1.204694e-02 -1.092769e-03  5.471276e-03 -3.384396e-03 -1.345570e-02
 [166]  1.271503e-02 -2.889772e-03  7.500112e-03  1.903173e-02  5.033518e-03
 [171] -8.142233e-03 -7.997227e-03 -9.421320e-03  1.322659e-02  2.528726e-03
 [176]  4.067168e-03  7.536574e-03 -6.546208e-03  1.880175e-02  2.140325e-02
 [181]  1.087602e-02  1.306478e-02  1.404132e-02  1.723769e-03 -1.477324e-03
 [186] -1.162363e-03 -4.053311e-04  2.657960e-02  3.153603e-03 -6.333766e-04
 [191] -8.580929e-03  2.560922e-02  2.587229e-02  2.087514e-02 -6.948128e-03
 [196] -5.784221e-02  3.335293e-02 -2.012096e-02 -1.391886e-03  2.798146e-02
 [201]  9.645771e-03  7.926365e-03  9.324933e-03  3.312128e-02  4.406874e-03
 [206] -5.877716e-03  2.008934e-02  1.110494e-03 -2.085979e-02  1.025997e-02
 [211] -1.696873e-02 -1.923690e-02 -1.890282e-02  3.379487e-02 -2.638978e-03
 [216] -3.658692e-02 -4.400439e-02  7.881052e-02  1.277399e-02 -2.898312e-02
 [221] -1.892563e-02  6.301114e-03 -1.235448e-02  4.797860e-02 -7.867665e-03
 [226]  2.072163e-02 -1.597739e-03  1.653180e-02  2.736973e-02  6.823890e-03
 [231] -5.755552e-03 -1.668966e-02  3.851510e-03  2.075767e-02  2.369666e-02
 [236] -5.454279e-04  4.953672e-03 -2.711761e-02  2.145655e-04 -7.607374e-03
 [241] -5.910369e-03 -3.050573e-02  6.137434e-03  5.952416e-03  1.802446e-02
 [246]  1.009836e-02  5.782052e-03  1.432483e-02 -1.431050e-02  2.551134e-03
 [251] -1.585380e-02 -9.138068e-03  2.042475e-04 -4.221638e-02 -1.186626e-02
 [256] -2.743481e-02  3.350043e-02 -9.954465e-03 -1.319888e-02  2.410206e-02

In [48]:
df_axp %>% filter(AXP.trade_profit>0) %>% select(AXP.trade_profit) %>% apply(2, sum)

AXP.trade_profit 
        2.275832

In [23]:
df_aapl <- apply_model('AAPL', pars)
df_all <- merge(df_axp, df_aapl, by=0)

In [24]:
head(df_all)

,Row.names,AXP.open,AXP.high,AXP.low,AXP.close,AXP.volume,AXP.adjusted,AXP.mean,AXP.predict,AXP.resid,⋯,AAPL.mean,AAPL.predict,AAPL.resid,AAPL.last_close,AAPL.high_resid,AAPL.low_resid,AAPL.max_resid,AAPL.hold,AAPL.trade_profit,AAPL.trade_times
,<I<chr>>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,2007-01-03,4.113820,4.125520,4.095178,4.100327,15.63074,3.853671,4.110349,4.110349,0.000000000,⋯,1.101079,1.101079,0.000000000,NA,0.02778495,-0.027784949,-0.02778495,0,0,0
2,2007-01-04,4.098171,4.103800,4.090838,4.093010,15.55091,3.846355,4.097319,4.110349,-0.013029792,⋯,1.109014,1.101079,0.007934699,1.096228,0.02048184,-0.004612440,0.02048184,0,0,0
3,2007-01-05,4.088494,4.092176,4.075841,4.079738,15.72773,3.833083,4.084008,4.103826,-0.019817799,⋯,1.113914,1.105048,0.008866198,1.118182,0.01941747,-0.001685073,0.01941747,0,0,0
4,2007-01-08,4.078046,4.090337,4.066459,4.089164,15.42499,3.842509,4.078398,4.091934,-0.013535620,⋯,1.121011,1.110368,0.010642981,1.111035,0.01791861,0.003367350,0.01791861,0,0,0
5,2007-01-09,4.092677,4.093511,4.077537,4.082778,15.41853,3.836123,4.085524,4.083604,0.001920440,⋯,1.156195,1.116918,0.039277214,1.115961,0.08326225,-0.004707818,0.08326225,0,0,0
6,2007-01-10,4.081597,4.083789,4.072610,4.082103,15.28954,3.835448,4.078200,4.084790,-0.006590347,⋯,1.227971,1.141177,0.086793973,1.195760,0.10954297,0.064044972,0.10954297,0,0,0


In [58]:
check_sum <- function(x) {
  
  if (x['AAPL.trade_profit'] != 0) {
    return(x['AAPL.trade_profit'])
  } else {
    return(x['AXP.trade_profit'])
  }
}


df_all %>% filter(AAPL.trade_profit!=0 | AXP.trade_profit!=0) %>% select(AAPL.trade_profit, AXP.trade_profit) %>% apply(1, check_sum) %>% sum

[1] 1.195338

In [34]:
df_all %>% filter(AAPL.hold==1 | AXP.hold==1) %>% select(AAPL.trade_profit, AXP.trade_profit)

AAPL.trade_profit,AXP.trade_profit
<dbl>,<dbl>
0.037522056,0.000000e+00
0.020993725,0.000000e+00
-0.059282819,0.000000e+00
-0.072792292,0.000000e+00
0.028773015,0.000000e+00
-0.036634668,0.000000e+00
0.000000000,7.486460e-03
0.000000000,1.129956e-02
0.000000000,-3.965227e-02


In [271]:
head(df1)

,open,high,low,close,volume,adjusted,mean,predict,resid,last_close,high_resid,low_resid,max_resid,hold,trade_profit,trade_times
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2007-01-03,1.125509,1.128864,1.073294,1.096228,20.93702,0.9382076,1.101079,1.101079,0.000000000,NA,0.02778495,-0.027784949,-0.02778495,0,0,0
2007-01-04,1.099207,1.121561,1.096467,1.118182,20.55752,0.9601606,1.109014,1.101079,0.007934699,1.096228,0.02048184,-0.004612440,0.02048184,0,0,0
2007-01-05,1.119465,1.124466,1.103363,1.111035,20.54263,0.9530141,1.113914,1.105735,0.008179428,1.118182,0.01873070,-0.002371843,0.01873070,0,0,0
2007-01-08,1.121678,1.128287,1.113735,1.115961,20.49650,0.9579400,1.121011,1.110752,0.010258822,1.111035,0.01753445,0.002983191,0.01753445,0,0,0
2007-01-09,1.127362,1.200180,1.112210,1.195760,21.93202,1.0377396,1.156195,1.117086,0.039109256,1.115961,0.08309429,-0.004875776,0.08309429,0,0,0
2007-01-10,1.219037,1.250720,1.205222,1.242507,21.80605,1.0844856,1.227971,1.141253,0.086718501,1.195760,0.10946750,0.063969500,0.10946750,0,0,0


In [272]:
df1 %>% result_summary

[1] 0.5287534

In [196]:
df1 %>% rename_all(function(x) paste0('AAPL.', x))

,AAPL.open,AAPL.high,AAPL.low,AAPL.close,AAPL.volume,AAPL.adjusted,AAPL.mean,AAPL.predict,AAPL.resid,AAPL.last_close,AAPL.high_resid,AAPL.low_resid,AAPL.max_resid,AAPL.hold,AAPL.trade_profit
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2007-01-03,1.125509,1.128864,1.073294,1.096228,20.93702,0.9382080,1.101079,1.101079,0.0000000000,NA,0.027784949,-0.027784949,-0.027784949,0,0
2007-01-04,1.099207,1.121561,1.096467,1.118182,20.55752,0.9601606,1.109014,1.101079,0.0079346993,1.096228,0.020481839,-0.004612440,0.020481839,0,0
2007-01-05,1.119465,1.124466,1.103363,1.111035,20.54263,0.9530141,1.113914,1.105735,0.0081794276,1.118182,0.018730699,-0.002371843,0.018730699,0,0
2007-01-08,1.121678,1.128287,1.113735,1.115961,20.49650,0.9579400,1.121011,1.110752,0.0102588219,1.111035,0.017534453,0.002983191,0.017534453,0,0
2007-01-09,1.127362,1.200180,1.112210,1.195760,21.93202,1.0377396,1.156195,1.117086,0.0391092564,1.115961,0.083094288,-0.004875776,0.083094288,0,0
2007-01-10,1.219037,1.250720,1.205222,1.242507,21.80605,1.0844856,1.227971,1.141253,0.0867185014,1.195760,0.109467503,0.063969500,0.109467503,0,0
2007-01-11,1.231519,1.240236,1.222725,1.230058,21.08808,1.0720376,1.231480,1.194846,0.0366340237,1.242507,0.045389719,0.027878329,0.045389719,0,0
2007-01-12,1.217347,1.222304,1.202865,1.217664,20.99534,1.0596438,1.212584,1.217487,-0.0049028255,1.230058,0.004816509,-0.014622160,-0.014622160,0,0
2007-01-16,1.228805,1.245080,1.226398,1.243537,20.94166,1.0855159,1.235739,1.214457,0.0212821377,1.217664,0.030623250,0.011941025,0.030623250,0,0
